In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
import requests
from api_keys import weather_api_key
import json
import time
import datetime

# Using pycountry to obtain full names of the countries
# source: https://pypi.org/project/pycountry/
import pycountry

# Adding options to display all of the rows of DataFrame w/o truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Range of latitudes and longitudes
range_of_latitude = (-90, 90)
range_of_longitude = (-180, 180)

output_path = os.path.join('..','Outputs')

In [6]:
# List for holding cities
cities = []
countries = []

# Create a set of random lat and lng combinations
random_latitudes = np.random.uniform(low = -90, high = 90, size=1500)
random_longitudes = np.random.uniform(low = -180, high = 180, size=1500)
random_geographic_locations = zip(random_latitudes, random_longitudes)

for a_location in random_geographic_locations:
    city = citipy.nearest_city(a_location[0], a_location[1]).city_name
    country = citipy.nearest_city(a_location[0], a_location[1]).country_code
    
    # If city is unique, then append it to city list along with full country name
    if city not in cities:
        cities.append(city)
        try:
            countries.append(pycountry.countries.get(alpha_2 = country.upper()).name)
        except:
            countries.append('N/A')

cities_and_countries = zip(cities,countries)

print(len(cities))
print(len(countries))

634
634


In [7]:
# base_url = "http://api.openweathermap.org/data/2.5/weather?"
base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial"

requests_per_minute = 0
elapsed_time_in_seconds = 0
record_number = 0
request_counter = 0

start_time = time.time()
print('Session started at: ' + time.ctime() + '\n')

# setup weather_df to append data
weather_df = pd.DataFrame({'City':[],\
                           'Country':[],\
                           'Country Code':[],\
                           'Date':[],\
                           'Latitude':[],\
                           'Longitude':[],\
                           'Max Temp':[],\
                           'Humidity':[],\
                           'Cloudiness':[],\
                           'Wind Speed':[]})

for _ in cities_and_countries:       
    a_city = _[0]
    a_country = _[1]       
    
    current_time = time.time()
    elapsed_time_in_seconds = round(current_time - start_time, 2)
    
    # Query OpenWeatherMap API and update counters
    query_url = base_url + '&APPID=' + weather_api_key + '&q=' + a_city
    response = requests.get(query_url)
    weather_response = response.json()
    
    request_counter = request_counter + 1
    
    record_number = record_number + 1
    if response:
        print(f'Record# {record_number}/{len(cities)} : {a_city.title()}, {a_country} : PROCESSED')
        
        try:
            response_items_df = pd.DataFrame([[a_city.title(),\
                                a_country,\
                                weather_response['sys']['country'],\
                                str(weather_response['dt']),\
                                weather_response['coord']['lat'],\
                                weather_response['coord']['lon'],\
                                weather_response['main']['temp_max'],\
                                weather_response['main']['humidity'],\
                                weather_response['clouds']['all'],\
                                weather_response['wind']['speed']]], columns=list(weather_df.columns))
                        
            weather_df = weather_df.append(response_items_df)
        except:
            pass
    else:
        print(f'Record# {record_number}/{len(cities)} : {a_city.title()}, {a_country} : NOT FOUND')
    
    if (elapsed_time_in_seconds <= 59.0 and request_counter >= 59):
        print('\nSession ended at: ' + time.ctime())
        print(f'{request_counter} requests made in {elapsed_time_in_seconds} secs')        
        print(f'\nWaiting for next minute to begin...')
        time.sleep(60.0 - elapsed_time_in_seconds)
        
        # Reset start_time and request_counter
        start_time = time.time()
        request_counter = 0
        print('\nSession continued at: ' + time.ctime())
        
#     if (request_counter == 20):
#         break

Session started at: Sat Feb  1 17:49:11 2020

Record# 1/634 : Vaini, Tonga : PROCESSED
Record# 2/634 : Grindavik, Iceland : PROCESSED
Record# 3/634 : Bermejo, Bolivia, Plurinational State of : PROCESSED
Record# 4/634 : Tocopilla, Chile : PROCESSED
Record# 5/634 : Arraial Do Cabo, Brazil : PROCESSED
Record# 6/634 : Odienne, Côte d'Ivoire : PROCESSED
Record# 7/634 : Iqaluit, Canada : PROCESSED
Record# 8/634 : Sawakin, Sudan : PROCESSED
Record# 9/634 : Nikolskoye, Russian Federation : PROCESSED
Record# 10/634 : Qaanaaq, Greenland : PROCESSED
Record# 11/634 : Pak Phanang, Thailand : PROCESSED
Record# 12/634 : Punta Arenas, Chile : PROCESSED
Record# 13/634 : Altay, China : PROCESSED
Record# 14/634 : Wilmington, United States : PROCESSED
Record# 15/634 : Barrow, United States : PROCESSED
Record# 16/634 : Shu, Kazakhstan : PROCESSED
Record# 17/634 : Hilo, United States : PROCESSED
Record# 18/634 : Ribeira Grande, Portugal : PROCESSED
Record# 19/634 : Puerto Ayora, Ecuador : PROCESSED
Record# 

Record# 154/634 : Dingle, Ireland : PROCESSED
Record# 155/634 : Saint George, Bermuda : PROCESSED
Record# 156/634 : Tiksi, Russian Federation : PROCESSED
Record# 157/634 : Rameshki, Russian Federation : PROCESSED
Record# 158/634 : Baruun-Urt, Mongolia : PROCESSED
Record# 159/634 : Fortuna, United States : PROCESSED
Record# 160/634 : Marcona, Peru : NOT FOUND
Record# 161/634 : Clyde River, Canada : PROCESSED
Record# 162/634 : Hirara, Japan : PROCESSED
Record# 163/634 : Castro, Chile : PROCESSED
Record# 164/634 : Salalah, Oman : PROCESSED
Record# 165/634 : Rio Gallegos, Argentina : PROCESSED
Record# 166/634 : Dordrecht, South Africa : PROCESSED
Record# 167/634 : Wanning, China : PROCESSED
Record# 168/634 : Husavik, Iceland : PROCESSED
Record# 169/634 : Victor Harbor, Australia : PROCESSED
Record# 170/634 : Kavieng, Papua New Guinea : PROCESSED
Record# 171/634 : Abu Samrah, Qatar : PROCESSED
Record# 172/634 : Amazar, Russian Federation : PROCESSED
Record# 173/634 : Colquechaca, Bolivia, P

Record# 303/634 : Andalan, Philippines : PROCESSED
Record# 304/634 : Sao Paulo De Olivenca, Brazil : PROCESSED
Record# 305/634 : Abyek, Iran, Islamic Republic of : PROCESSED
Record# 306/634 : Saint-Pierre, Réunion : PROCESSED
Record# 307/634 : Port Lincoln, Australia : PROCESSED
Record# 308/634 : Pathein, Myanmar : PROCESSED
Record# 309/634 : Teya, Russian Federation : PROCESSED
Record# 310/634 : Kribi, Cameroon : PROCESSED
Record# 311/634 : Provideniya, Russian Federation : PROCESSED
Record# 312/634 : Katsuura, Japan : PROCESSED
Record# 313/634 : Bridlington, United Kingdom : PROCESSED
Record# 314/634 : Marsh Harbour, Bahamas : PROCESSED
Record# 315/634 : Kurmanayevka, Russian Federation : PROCESSED
Record# 316/634 : Waddan, Libya : PROCESSED
Record# 317/634 : Tongchuan, China : PROCESSED
Record# 318/634 : Ilulissat, Greenland : PROCESSED
Record# 319/634 : Sao Joao Da Barra, Brazil : PROCESSED
Record# 320/634 : Praia Da Vitoria, Portugal : PROCESSED
Record# 321/634 : Fagersta, Sweden 

Record# 459/634 : Sao Jose Da Coroa Grande, Brazil : PROCESSED
Record# 460/634 : Kulhudhuffushi, Maldives : PROCESSED
Record# 461/634 : Bonthe, Sierra Leone : PROCESSED
Record# 462/634 : Uhingen, Germany : PROCESSED
Record# 463/634 : Vanimo, Papua New Guinea : PROCESSED
Record# 464/634 : Botngard, Norway : PROCESSED
Record# 465/634 : Kangaba, Mali : PROCESSED
Record# 466/634 : Shiyan, China : PROCESSED
Record# 467/634 : Kudahuvadhoo, Maldives : PROCESSED
Record# 468/634 : Huarmey, Peru : PROCESSED
Record# 469/634 : Bontang, Indonesia : PROCESSED
Record# 470/634 : Sao Gabriel Da Cachoeira, Brazil : PROCESSED
Record# 471/634 : Broken Hill, Australia : PROCESSED
Record# 472/634 : Mezen, Russian Federation : PROCESSED

Session ended at: Sat Feb  1 17:56:19 2020
59 requests made in 7.2 secs

Waiting for next minute to begin...

Session continued at: Sat Feb  1 17:57:12 2020
Record# 473/634 : Vaitape, French Polynesia : PROCESSED
Record# 474/634 : Darhan, Mongolia : PROCESSED
Record# 475/634

Record# 610/634 : Talakan, Russian Federation : PROCESSED
Record# 611/634 : Bougouni, Mali : PROCESSED
Record# 612/634 : Katobu, Indonesia : PROCESSED
Record# 613/634 : Verkhnevilyuysk, Russian Federation : PROCESSED
Record# 614/634 : Tahta, Egypt : PROCESSED
Record# 615/634 : Nguiu, Australia : NOT FOUND
Record# 616/634 : Kargasok, Russian Federation : PROCESSED
Record# 617/634 : Korla, China : PROCESSED
Record# 618/634 : Leningradskiy, Russian Federation : PROCESSED
Record# 619/634 : Maloy, Norway : PROCESSED
Record# 620/634 : Tatawin, Tunisia : PROCESSED
Record# 621/634 : Escuinapa, Mexico : PROCESSED
Record# 622/634 : San Cristobal, Ecuador : PROCESSED
Record# 623/634 : Beloha, Madagascar : PROCESSED
Record# 624/634 : Kushima, Japan : PROCESSED
Record# 625/634 : Yakeshi, China : PROCESSED
Record# 626/634 : Zhuhai, China : PROCESSED
Record# 627/634 : Namatanai, Papua New Guinea : PROCESSED
Record# 628/634 : Skjervoy, Norway : PROCESSED
Record# 629/634 : Sosva, Russian Federation : P

In [8]:
weather_df = weather_df.reset_index(drop=True)
print(weather_df.count())
weather_df.head(20)

City            578
Country         578
Country Code    578
Date            578
Latitude        578
Longitude       578
Max Temp        578
Humidity        578
Cloudiness      578
Wind Speed      578
dtype: int64


,City,Country,Country Code,Date,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
0,Vaini,Tonga,TO,1580600753,-21.20,-175.20,84.20,70.0,40.0,8.05
1,Grindavik,Iceland,IS,1580600944,63.84,-22.43,28.40,86.0,75.0,10.29
2,Bermejo,"Bolivia, Plurinational State of",AR,1580600945,-32.88,-68.80,95.00,21.0,40.0,4.70
3,Tocopilla,Chile,CL,1580600945,-22.09,-70.20,60.04,57.0,3.0,3.00
4,Arraial Do Cabo,Brazil,BR,1580600865,-22.97,-42.03,80.11,85.0,88.0,10.36
5,Odienne,Côte d'Ivoire,CI,1580600945,9.51,-7.57,71.22,16.0,9.0,1.43
6,Iqaluit,Canada,CA,1580600945,63.75,-68.51,12.20,78.0,90.0,8.05
7,Sawakin,Sudan,SD,1580600945,19.11,37.33,72.01,75.0,61.0,15.30
8,Nikolskoye,Russian Federation,RU,1580600945,59.70,30.79,33.80,96.0,90.0,11.18
9,Qaanaaq,Greenland,GL,1580600946,77.48,-69.36,-24.25,84.0,97.0,6.22


In [9]:
weather_df.to_csv(output_path+'/weather_df.csv', index = False)